# *Bagging* y *Boosting* para un problema de regresión

A continuación vamos a trabajar con el Data Set *Fish Market* que contiene información referente a las dimensiones de varias especies de pescado que se venden de forma habitual en el mercado. Vamos a comenzar realizando la carga del fichero .csv mediante ``pandas``.

https://www.kaggle.com/datasets/aungpyaeap/fish-market

In [1]:
import pandas as pd

df = pd.read_csv('Fish.csv')
df.head()

,Species,Weight,Length1,Length2,Length3,Height,Width
0,Bream,242.0,23.2,25.4,30.0,11.5200,4.0200
1,Bream,290.0,24.0,26.3,31.2,12.4800,4.3056
2,Bream,340.0,23.9,26.5,31.1,12.3778,4.6961
3,Bream,363.0,26.3,29.0,33.5,12.7300,4.4555
4,Bream,430.0,26.5,29.0,34.0,12.4440,5.1340


Podemos observar como este Data Set cuenta con 7 columnas. Para nuestro experimento vamos a tomar la columna ``Weight`` como variable dependiente, y el resto como variables independientes. Realizaremos una regresión lineal múltiple con el objetivo de poder predecir el peso de un patrón a partir de los valores de sus dimensiones y su especie. 

A continuación vamos a separar el conjunto de datos en variables de entrada y variable objetivo.

In [2]:
X = df[['Species', 'Length1', 'Length2', 'Length3', 'Height', 'Width']]
Y = df['Weight']

print(f'Dimensiones de entrada -> {X.shape}')
print(f'Dimensiones de variable objetivo -> {Y.shape}')

Dimensiones de entrada -> (159, 6)
Dimensiones de variable objetivo -> (159,)


In [3]:
X['Species'].value_counts()

Perch        56
Bream        35
Roach        20
Pike         17
Smelt        14
Parkki       11
Whitefish     6
Name: Species, dtype: int64

Un paso adicional antes de proceder con la fase de entrenamiento consistiría en aplicar un escalado de los datos y transformar la variable ``Species`` en numérica.

In [4]:
X = pd.get_dummies(X)
X.head()

,Length1,Length2,Length3,Height,Width,Species_Bream,Species_Parkki,Species_Perch,Species_Pike,Species_Roach,Species_Smelt,Species_Whitefish
0,23.2,25.4,30.0,11.5200,4.0200,1,0,0,0,0,0,0
1,24.0,26.3,31.2,12.4800,4.3056,1,0,0,0,0,0,0
2,23.9,26.5,31.1,12.3778,4.6961,1,0,0,0,0,0,0
3,26.3,29.0,33.5,12.7300,4.4555,1,0,0,0,0,0,0
4,26.5,29.0,34.0,12.4440,5.1340,1,0,0,0,0,0,0


In [5]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import random, time

seed = random.seed(time.time())
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.3, random_state = seed)

ss = StandardScaler()
X_train = ss.fit_transform(X_train)
X_test = ss.transform(X_test)

Una vez aplicadas las transformaciones convenientes y separados los conjuntos de test y entrenamiento, vamos a proceder a la fase de experimentación. En concreto, vamos a realizar la regresión descrita previamente utilizando el modelo base, un *ensemble* con *Bagging* y otro *ensemble* con *Boosting*.

In [6]:
from sklearn.ensemble import AdaBoostRegressor, BaggingRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression
#from sklearn.tree import DecisionTreeRegressor
import numpy as np

base_model = LinearRegression()

# Ajuste con el modelo base
base_model.fit(X_train, y_train)
y_pred_1 = base_model.predict(X_test)
rmse_1 = np.sqrt(mean_squared_error(y_test, y_pred_1))

# Ajuste con Bagging
model_bag = BaggingRegressor(base_estimator = base_model, n_estimators = 100, 
                            max_samples = 0.1, random_state = seed)
model_bag.fit(X_train, y_train)
y_pred_2 = model_bag.predict(X_test)
rmse_2 = np.sqrt(mean_squared_error(y_test, y_pred_2))

# Ajuste con Boosting
model_boost = AdaBoostRegressor(base_estimator = base_model, n_estimators = 100, 
                                random_state = seed)
model_boost.fit(X_train, y_train)
y_pred_3 = model_boost.predict(X_test)
rmse_3 = np.sqrt(mean_squared_error(y_test, y_pred_3))

print(f'RMSE con modelo base: {rmse_1}')
print(f'R^2 con modelo base {r2_score(y_test, y_pred_1)}')
print('----------------------------------------------------------')
print(f'RMSE con Bagging: {rmse_2}')
print(f'R^2 con Bagging {r2_score(y_test, y_pred_2)}')
print('----------------------------------------------------------')
print(f'RMSE con Boosting {rmse_3}')
print(f'R^2 con Boosting {r2_score(y_test, y_pred_3)}')


RMSE con modelo base: 104.61887570908858
R^2 con modelo base 0.91898148175618
----------------------------------------------------------
RMSE con Bagging: 148.34644745257935
R^2 con Bagging 0.8371009694029896
----------------------------------------------------------
RMSE con Boosting 106.74704648758177
R^2 con Boosting 0.9156517776365352


En este caso, al contrario de lo que se podría haber razonado en un principio, los modelos con *boosting* y *bagging* ofrecen un rendimiento peor que el modelo base (especialmente notable en el caso de *bagging*). Esto es así porque los modelos de varianza baja no se suelen beneficiar de este tipo de estrategias de entrenamiento.

Un modelo con baja varianza es aquel que tiende a generar modelos más simples y evitar el *overfitting*. Un ejemplo de modelo de baja varianza podría ser la regresión lineal que nos indica la "tendencia" de los datos, pero no se ajusta perfectamente a ellos. Un ejemplo de modelo con alta varianza podrían ser los árboles de decisión que, cuando no se les aplica alguna estrategia de poda, generan un alto número de nodos "puros" con tendencia a sobreajustarse a los datos.

En este caso concreto, el modelo base ofrece un coeficiente de correlación $R^{2}$ ~0.90 por lo que resulta adecuado para realizar predicciones de posibles nuevas entradas. Los modelos de *bagging* y *boosting* generan cierto número de estimadores (en este caso hemos fijado ``n_estimators`` a 100) y finalmente calculan el resultado de la predicción como una media de las salidas de cada estimador (media aritética en *bagging* y ponderada en *boosting*). Sin embargo, en *bagging* sabemos que los puntos se escogen con reemplazo, es decir, pueden existir puntos duplicados en varios estimadores. Este duplicado puede hacer que ciertos puntos tengan más influencia que otros a la hora de realizar predicciones y, en consecuencia, que las líneas de regresión se alejen de las que habríamos obtenido con el modelo base.

Por tanto, las técnicas de ensemble resultan inadecuadas para modelos con varianza baja y es habitual obtener resultados ligeramente peores que aplicando el modelo sin ensemble. 

# *Bagging* y *Boosting* para un problema de clasificación

Para el apartado de clasificación de esta actividad, vamos a hacer uso del *Pima Indians Diabetes Database* que determina si un paciente padece o no diabetes en funcion de ciertos parámetros de salud. Utilizaremos ``pandas`` y su función ``read_csv()`` para realizar la lectura del fichero.

https://www.kaggle.com/datasets/uciml/pima-indians-diabetes-database

In [7]:
df_d = pd.read_csv('./diabetes.csv', header = 0)
df_d.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [8]:
df_d['Outcome'].value_counts()

0    500
1    268
Name: Outcome, dtype: int64

De nuevo, nos enfrentamos a un dataset notablemente desequilibrado, pues existen muchas más instancias de la clase "0" (no diabetes) que de la clase "1" (diabetes). Utilizaremos la ``balanced_accuracy`` como métrica del rendimiento de los modelos que generemos pues, como hemos visto en numerosas ocasiones, es robusta frente a data sets desequilibrados.

In [9]:
df_d.dtypes

Pregnancies                   int64
Glucose                       int64
BloodPressure                 int64
SkinThickness                 int64
Insulin                       int64
BMI                         float64
DiabetesPedigreeFunction    float64
Age                           int64
Outcome                       int64
dtype: object

El data set cuenta únicamente con entradas numéricas, por lo que no es necesario reemplazar ninguna variable categórica mediante *one-hot encoding*. En este caso vamos a hacer uso de ``DecisionTreeClassifier`` como modelo base y, por tanto, no se hace necesario realizar ningún escalado o normalizado de los datos (pues este modelo no se basa en distancias). Procedemos a continuación a separar nuestro data set en conjunto de entrenamiento y conjunto de test.

In [10]:
X = df_d[df_d.columns[:-1]]
Y = df_d['Outcome']

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.3, random_state = seed)

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(537, 8)
(231, 8)
(537,)
(231,)


In [11]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier, BaggingClassifier
from sklearn.metrics import balanced_accuracy_score

max_depths = list(range(3, 15)) + [None]
baccs = []

for max_depth in max_depths:

    base_model = DecisionTreeClassifier(max_depth = max_depth)

    # Ajuste con el modelo base
    base_model.fit(X_train, y_train)
    y_pred_1 = base_model.predict(X_test)
    ba_1 = balanced_accuracy_score(y_test, y_pred_1)

    # Ajuste con Bagging
    model_bag = BaggingClassifier(base_estimator = base_model, n_estimators = 100, 
                                max_samples = 0.1, random_state = seed)
    model_bag.fit(X_train, y_train)
    y_pred_2 = model_bag.predict(X_test)
    ba_2 = balanced_accuracy_score(y_test, y_pred_2)

    # Ajuste con Boosting
    model_boost = AdaBoostClassifier(base_estimator = base_model, n_estimators = 100, 
                                    random_state = seed)
    model_boost.fit(X_train, y_train)
    y_pred_3 = model_boost.predict(X_test)
    ba_3 = balanced_accuracy_score(y_test, y_pred_3)

    baccs.append([ba_1, ba_2, ba_3])


In [12]:
baccs_aux = np.array(baccs)
pos1, pos2 = np.unravel_index(baccs_aux.argmax(), baccs_aux.shape)

words = ['Modelo Base', 'Bagging', 'Boosting']
print(f'Rendimiento óptimo con: \n\t-Balanced Accuracy = {baccs_aux[pos1][pos2]}\n\t-max_depth = {max_depths[pos1]}\n\t-{words[pos2]}\n')

maxes = np.argmax(baccs_aux, axis = 0)
print(f'Mejor rendimiento con Modelo Base\n\t-Balanced Accuracy = {baccs_aux[maxes[0]][0]}\n\t-max_depth = {max_depths[maxes[0]]}\n')
print(f'Mejor rendimiento con Bagging\n\t-Balanced Accuracy = {baccs_aux[maxes[1]][1]}\n\t-max_depth = {max_depths[maxes[1]]}\n')
print(f'Mejor rendimiento con Boosting\n\t-Balanced Accuracy = {baccs_aux[maxes[2]][2]}\n\t-max_depth = {max_depths[maxes[2]]}\n')

Rendimiento óptimo con: 
	-Balanced Accuracy = 0.7851851851851852
	-max_depth = 9
	-Boosting

Mejor rendimiento con Modelo Base
	-Balanced Accuracy = 0.7708641975308642
	-max_depth = 4

Mejor rendimiento con Bagging
	-Balanced Accuracy = 0.7766666666666666
	-max_depth = 8

Mejor rendimiento con Boosting
	-Balanced Accuracy = 0.7851851851851852
	-max_depth = 9



Para este problema particular vemos como el modelo entrenado mediante *Boosting* parece ofrecer un rendimiento superior. Sin embargo, la diferencia entre modelos no es demasiado significativa (~2% entre *Bagging* y *Boosting* y ~4% entre modelo base y *Boosting*). El mejor rendimiento de *boosting* podría atribuirse a la optimización entre predictores al dar un mayor peso a aquellas instancias mal clasificadas. 

De nuevo, estas diferencias no parecen demasiado significativas para este problema concreto. En general, modelos que tienden al *overfitting* como los *decision trees* se benefician de *bagging* en lugar de *boosting* ya que este último suele encontrar un mayor ajuste sobre el conjunto de datos. Por tanto, es posible que para otro tipo de problemas, el modelo de *decision tree* se beneficiase de una técnica de *bagging*.